In [1]:
import pandas as pd
df = pd.read_csv("testdata/liberia.csv")
df.drop(["quality", "problems"], axis=1, inplace=True)
df[:1000].to_csv("testdata/liberia_noq.csv")

In [2]:
len(df)

11724

In [3]:
import holoclean
from detect import *
from repair.featurize import *

## 1. Setup
Setup holoclean session

In [4]:
hc = holoclean.HoloClean(
    db_name='holo',
    domain_thresh_1=0.0,
    domain_thresh_2=0.0,
    weak_label_thresh=0.99,
    max_domain=10000,
    cor_strength=0.6,
    nb_cor_strength=0.8,
    weight_decay=0.01,
    learning_rate=0.001,
    threads=1,
    batch_size=1,
    verbose=True,
    timeout=3 * 60000,
    print_fw=True,
).session

06:44:25 - [DEBUG] - initiating session with parameters: {'db_user': 'holocleanuser', 'db_pwd': 'abcd1234', 'db_host': 'localhost', 'db_name': 'holo', 'threads': 1, 'timeout': 180000, 'seed': 45, 'layer_sizes': [1], 'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 20, 'weight_decay': 0.01, 'momentum': 0.0, 'batch_size': 1, 'weak_label_thresh': 0.99, 'domain_thresh_1': 0.0, 'domain_thresh_2': 0.0, 'max_domain': 10000, 'cor_strength': 0.6, 'nb_cor_strength': 0.8, 'feature_norm': False, 'weight_norm': False, 'estimator_type': 'NaiveBayes', 'estimator_epochs': 10, 'estimator_batch_size': 32, 'estimator_embedding_size': 10, 'train_attrs': None, 'infer_mode': 'dk', 'verbose': True, 'bias': False, 'print_fw': True, 'debug_mode': False}


## 2. Load training data and denial constraints

Load data

In [5]:
hc.load_data("liberia", "testdata/liberia_noq.csv")

06:44:25 - [WARNING] - Dropping the following null column from the dataset: 'Region'
06:44:25 - [WARNING] - Dropping the following null column from the dataset: 'Stroke test'
06:44:25 - [ INFO] - Loaded 1000 rows with 19000 cells
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [DEBUG] - Time to

Constrains

constrain_path = "testdata/liberia_constraints.txt"

!cat $constrain_path

hc.load_dcs(constrain_path)
hc.ds.set_constraints(hc.get_dcs())

## 3. Detectors

In [6]:
detectors = [NullDetector()]
hc.detect_errors(detectors)

06:44:25 - [DEBUG] - DONE with Error Detector: NullDetector in 0.04 secs
06:44:25 - [ INFO] - detected 483 potentially erroneous cells
06:44:25 - [DEBUG] - Time to create index: 0.00 secs
06:44:25 - [ INFO] - DONE with error detection.
06:44:25 - [DEBUG] - Time to detect errors: 0.08 secs


## 4. Repair

In [7]:
hc.generate_domain()

06:44:25 - [DEBUG] - Computing correlations...
06:44:32 - [DEBUG] - correlations:
attr             Unnamed: 0  Instance  Submission Date  Submitter  settlement  \
cond_attr                                                                       
CMMoney            0.178756  0.178756         0.182726   0.168557    0.192597   
Functionality      0.109320  0.109320         0.111965   0.087271    0.113627   
Geo Code           0.988325  0.988325         0.995203   0.998560    0.997882   
Instance           1.000000  1.000000         1.000000   1.000000    1.000000   
Latitude           0.993098  0.993098         1.000000   1.000000    1.000000   
Longitude          0.993098  0.993098         1.000000   1.000000    1.000000   
Submission Date    0.992775  0.992775         1.000000   0.999674    0.999793   
Submitter          0.585167  0.585167         0.590139   1.000000    0.613940   
Unnamed: 0         1.000000  1.000000         1.000000   1.000000    1.000000   
Wptype             0.026905

In [8]:
hc.run_estimator()

06:44:37 - [DEBUG] - training estimator for estimating domain value probabilities...
06:44:37 - [DEBUG] - using estimator: NaiveBayes
06:44:37 - [DEBUG] - DONE training estimator in 0.01s
06:44:37 - [DEBUG] - predicting domain value probabilities from estimator...
06:44:37 - [DEBUG] - DONE predictions in 0.00 secs, re-constructing cell domain...
06:44:37 - [DEBUG] - re-assembling final cell domain table...
100%|██████████| 6970/6970 [00:00<00:00, 142325.85it/s]
06:44:39 - [DEBUG] - DONE assembling cell domain table in 2.52s
06:44:39 - [ INFO] - number of (additional) weak labels assigned from estimator: 144
06:44:39 - [DEBUG] - DONE generating domain and weak labels
06:44:40 - [DEBUG] - Time to create index: 0.00 secs
06:44:40 - [DEBUG] - Time to create index: 0.00 secs
06:44:40 - [DEBUG] - Time to create index: 0.00 secs
06:44:40 - [DEBUG] - Time to create table: 0.00 secs
06:44:40 - [DEBUG] - Time to create index: 0.00 secs


In [9]:
featurizers = [FreqFeaturizer(), OccurAttrFeaturizer()]
hc.repair_errors(featurizers)

06:44:40 - [DEBUG] - Time to execute query: 0.00 secs
06:44:40 - [DEBUG] - Time to execute query: 0.00 secs
06:44:40 - [DEBUG] - Time to execute query: 0.00 secs
06:44:40 - [DEBUG] - featurizing training data...
06:44:40 - [DEBUG] - Time to execute query: 0.01 secs
100%|██████████| 6970/6970 [00:12<00:00, 559.48it/s]
06:44:54 - [DEBUG] - DONE featurization. Feature tensor size: torch.Size([6970, 80, 144])
06:44:54 - [DEBUG] - generating weak labels...
06:44:54 - [DEBUG] - Time to execute query: 0.00 secs
100%|██████████| 6661/6661 [00:00<00:00, 71341.86it/s]
06:44:54 - [DEBUG] - DONE generating weak labels.
06:44:54 - [DEBUG] - generating mask...
06:44:54 - [DEBUG] - Time to execute query: 0.00 secs
100%|██████████| 6970/6970 [00:00<00:00, 103917.20it/s]
06:44:54 - [DEBUG] - DONE generating mask.
06:44:54 - [ INFO] - DONE setting up featurized dataset.
06:44:54 - [DEBUG] - Time to featurize data: 13.99 secs
06:44:54 - [DEBUG] - training model with first layer size: ['torch.Size([18, 1]

'featurizer FreqFeaturizer,size 18,max 0.0073,min -0.2639,avg -0.0143,abs_avg 0.0152,weights:\nUnnamed: 0 -0.0\nInstance -0.0\nSubmission Date 0.0\nSubmitter -0.0\nsettlement 0.0\nWptype 0.0\nhandpump 0.0\nFunctionality -0.0\nYearCons 0.007\npartner -0.001\ndrinking? 0.0\nquantity -0.0\ncommittee 0.0\nCMMoney -0.264\nphoto 0.0\nLatitude 0.0\nLongitude 0.0\nGeo Code 0.0\nfeaturizer OccurAttrFeaturizer,size 126,max 1.2106,min -0.2599,avg 0.1930,abs_avg 0.2070,weights:\nCMMoney | Unnamed: 0 0.262\nCMMoney | Instance 0.262\nCMMoney | Submission Date 0.262\nCMMoney | Submitter -0.166\nCMMoney | settlement 0.257\nCMMoney | Wptype -0.26\nCMMoney | handpump -0.26\nCMMoney | Functionality 0.505\nCMMoney | YearCons 0.514\nCMMoney | partner 0.642\nCMMoney | drinking? -0.201\nCMMoney | quantity 0.344\nCMMoney | committee 0.046\nCMMoney | CMMoney 0.004\nCMMoney | photo 0.921\nCMMoney | Latitude 0.916\nCMMoney | Longitude 0.928\nCMMoney | Geo Code 0.274\nGeo Code | Unnamed: 0 0.485\nGeo Code | Insta

In [13]:
df2 = hc.ds.get_raw_data()

In [16]:
df.equals(df)

True